# Data Ingestion

### Arctic
[Arctic](http://github.com/manahl/arctic) is a "High performance datastore for time series and tick data", built by the rather large hedge fund Man AHL and built on MongoDB. Generally speaking, I have avoided MongoDB in the past because, well, [it doesn't do what it says on the tin](https://aphyr.com/posts/322-jepsen-mongodb-stale-reads). This seems to have been [fixed](https://jepsen.io/analyses/mongodb-3-4-0-rc3), and in fairness, it's not too important if you lose a handful out of a few billion rows of timeseries data.

Arctic expects to be able to read port '27017'

In [3]:
from arctic import TICK_STORE
from arctic import Arctic

arctic_db = Arctic('localhost')
arctic_lib = arctic_db.initialize_library("FOREX", lib_type=TICK_STORE)

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused